# Concurrency and Parallelism in Python


## CPU
* parallel execution - tasks execute on multiple CPUs at the same time
* concurrent execution - only one task runs at one time on a single CPU, however, making progress simultaneously

## Memory
* each process created by a Parent process has its separate memory
* threads spawned by a process 
  * share memory space and other OS resources with other threads, consuming less memory than processes do.
    * code section
    * data section
  * each thread has its own 
    * thread ID, 
    * a program counter, 
    * a register set, 
    * and a stack. 

## Inter-Process Communication and Locks
* Since **processes** have different memory segments, a _communication channel_ is required to pass data between them
* **Threads** - because of shared resources, we need to use _locks_ to avoid the _race condition_. A race condition occurs when shared data is accessed and manipulated concurrently. However, using locks make sure that only one thread manipulates the data at one time. For a thread to manipulate the shared data, it has to acquire the lock. During that time, if some other thread tries to acquire the lock, it will have to wait until the lock gets released.

## GIL
Programs in Python are single-threaded and use a single CPU because of the Global Interpreter Lock or GIL. It is a lock that only allows one thread to hold control of the Python interpreter, and thus only one thread gets executed at a time. Therefore, Python cannot use multiprocessing automatically. However, the multiprocessing module solves this problem by bypassing the GIL.

In multiprocessing, each process has a separate GIL and instance of a Python interpreter. However, in multithreading, all the threads have a single GIL and thus one Python interpreter. Therefore, only one thread can be executed at one time.

## Performance
* I/O bound task - utilizes most of its time performing I/O operations
* CPU bound task - requires the CPU mostly to complete it

In [1]:
import functions as f
import time

In [2]:
# CPU-bound task
def cpu_bound(x):
    count = 0
    for i in range(0, x**x):
      count += 1
    return count
cpu_bound(3)

27

In [19]:
# Multi-processing
from multiprocessing import Process

starttime = time.time()
processlist = []
for i in range(0, 4):
    process = Process(target=f.cpu_bound, args=(9,))
    processlist.append(process)
    process.start()

for process in processlist:
    process.join()
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

Time taken 20.169926166534424 seconds


In [4]:
# Multi-processing - another way
from multiprocessing import Pool

starttime = time.time()
with Pool(4) as p:
    print(p.map(f.cpu_bound, [9, 9, 9, 9]))
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

[387420489, 387420489, 387420489, 387420489]
Time taken 21.96160912513733 seconds


In [6]:
# Single Processing and Threading
starttime = time.time()
for i in range(0, 4):
    f.cpu_bound(9)
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

Time taken 76.61669826507568 seconds


In [8]:
# Multi-threading
import threading

starttime = time.time()
threads = []
for i in range(0, 4):
    thread = threading.Thread(target=f.cpu_bound, args=(9,))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

Time taken 74.9468080997467 seconds


## CPU Bound Tasks - Conclusions
* multiprocessing < serial execution <= multithreading
  * in multiprocessing, processes run in parallel and utilize multiple CPUs. Since our code requires CPU most of the time, multiprocessing gives better results. 
  * However, in multithreading, only one thread runs at one time. Because there is no blocking due to any I/O operation, the CPU does not remain idle. Hence, it almost gives the same performance as serial execution does.
  * there is also an overhead of switching between the threads, which is not in serial execution. Thus, serial execution performs better than multithreading

In [2]:
# I/O-bound task
def io_bound(i):
    file_name = "file" + str(i) + ".txt"
    f = open(file_name, "w")
    print("writing to a file")
    for j in range(0, 8 ** 8):
        f.write("This is a sample text")

In [3]:
# Multi-processing
from multiprocessing import Process

starttime = time.time()
processlist = []
for i in range(0, 4):
    process = Process(target=f.io_bound, args=(i,))
    processlist.append(process)
    process.start()

for process in processlist:
    process.join()
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

writing to a file
writing to a file
writing to a file
writing to a file
Time taken 3.2585530281066895 seconds


In [7]:
# Uniprocessing and Single threading
starttime = time.time()
for i in range(0, 4):
    f.io_bound(i)
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

writing to a file
writing to a file
writing to a file
writing to a file
Time taken 8.330154180526733 seconds


In [8]:
# Multi-threading
starttime = time.time()
threads = []
for i in range(0, 4):
    thread = threading.Thread(target=f.io_bound, args=(i,))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
endtime = time.time()
print(f"Time taken {endtime-starttime} seconds")

writing to a file
writing to a file
writing to a file
writing to a file
Time taken 7.467015027999878 seconds


## I/O-Bound Tasks Conclusions
* When an I/O operation comes up in serial execution, the CPU remains idle and that thread blocks. Hence, no activity is done on the part of that process until that operation gets completed. 
* However, in multithreading, GIL releases the lock, and some other thread acquires it and starts running. That is why multithreading performs better than serial execution. Moreover, the overhead of creating a process is more than creating a thread. Therefore, multithreading performs better than multiprocessing, in this case.

**So, use multithreading when tasks include I/O operations or network requests mostly, and use multiprocessing when you have CPU intensive tasks**

<table>
    <tr>
        <th>Multi-threading</th>
        <th>Multi-processing</th>
    </tr>
    <tr>
        <td>Only uses a single CPU</td>
        <td>Uses multiple CPUs or cores</td>
    </tr>
    <tr>
        <td>Thread creation is faster than process creation, i.e., less overhead</td>
        <td>Process creation is slower</td>
    </tr>
    <tr>
        <td>Concurrent Execution</td>
        <td>Parallel Execution</td>
    </tr>
    <tr>
        <td>Threads have the same memory space</td>
        <td>Processes have a separate memory space</td>
    </tr>
    <tr>
        <td>Requires locks to handle shared data</td>
        <td>It does not require locks as threads do because the memory space is different unless you explicitly use some shared resource</td>
    </tr>
    <tr>
        <td>One GIL</td>
        <td>Each process has a separate GIL</td>
    </tr>
    <tr>
        <td>No communication channel is required</td>
        <td>Processes require a communication channel for IPC</td>
    </tr>
    <tr>
        <td>Context switching between threads is faster than processes</td>
        <td>Context switching is slower</td>
    </tr>
    <tr>
        <td>Suitable for I/O bound and network bound tasks</td>
        <td>Suitable for CPU bound tasks</td>
    </tr>
<table>

## Coroutines and asyncio

**single-threaded, single-process design: it uses cooperative multitasking**

* Asynchronous routines are able to “pause” while waiting on their ultimate result and let other routines run in the meantime.
* Asynchronous code facilitates concurrent execution - gives the look and feel of concurrency

![title](img/asyncio.webp)

Chess master Judit Polgár hosts a chess exhibition in which she plays multiple amateur players. She has two ways of conducting the exhibition: synchronously and asynchronously.

Assumptions:

24 opponents
Judit makes each chess move in 5 seconds
Opponents each take 55 seconds to make a move
Games average 30 pair-moves (60 moves total)
Synchronous version: Judit plays one game at a time, never two at the same time, until the game is complete. Each game takes (55 + 5) * 30 == 1800 seconds, or 30 minutes. The entire exhibition takes 24 * 30 == 720 minutes, or 12 hours.

Asynchronous version: Judit moves from table to table, making one move at each table. She leaves the table and lets the opponent make their next move during the wait time. One move on all 24 games takes Judit 24 * 5 == 120 seconds, or 2 minutes. The entire exhibition is now cut down to 120 * 30 == 3600 seconds, or just 1 hour.

**Async IO** takes long waiting periods in which functions would otherwise be blocking and allows other functions to run during that downtime. (A function that blocks effectively forbids others from running from the time that it starts until the time that it returns.)

**coroutine** is a specialized version of a Python generator function (function that can suspend its execution before reaching return, and it can indirectly pass control to another coroutine for some time)

In [25]:
import asyncio

async def async_count():  # native coroutine
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def async_main():
    await asyncio.gather(count(), count(), count())

s = time.perf_counter()
await async_main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
Executed in 1.00 seconds.


In [24]:
def sync_count():
    print("One")
    time.sleep(1)
    print("Two")

def sync_main():
    for _ in range(3):
        sync_count()

s = time.perf_counter()
sync_main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

One
Two
One
Two
One
Two
Executed in 3.01 seconds.


In [30]:
def threading_count():
    print("One")
    time.sleep(1)
    print("Two")

def threading_main():
    thread1 = threading.Thread(target=threading_count)
    thread2 = threading.Thread(target=threading_count)
    thread3 = threading.Thread(target=threading_count)
    
    thread1.start()
    thread2.start()
    thread3.start()
    
    thread1.join()
    thread2.join()
    thread3.join()

s = time.perf_counter()
threading_main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

One
One
One
TwoTwo

Two
Executed in 1.01 seconds.


## Multi-threading vs asyncio

In [34]:
# Sequential code
def sync_api1():
    print("call 1st API")
    time.sleep(4)
    print("1st API finish")
  
def sync_api2():
    print("call 2st API")
    time.sleep(3)
    print("2nd API finish")

start = time.time()
sync_api1()
sync_api2()
print(time.time()-start)

call 1st API
1st API finish
call 2st API
2nd API finish
7.005630016326904


In [35]:
# Multi-threaded code

def threaded_api1():
    print("call 1st API")
    time.sleep(4)
    print("1st API finish")
  
def threaded_api2():
    print("call 2st API")
    time.sleep(3)
    print("2nd API finish")

start = time.time()

# create threads
t1 = threading.Thread(target=threaded_api1, name='t1')
t2 = threading.Thread(target=threaded_api2, name='t2')  

# start threads
t1.start()
t2.start()

# wait untill all threads finished
t1.join()
t2.join()

print(time.time()-start)

call 1st API
call 2st API
2nd API finish
1st API finish
4.006623268127441


In [42]:
# Async IO = non-blocking IO, using event loop
async def async_api1():
    print("call 1st API")
    await asyncio.sleep(4)
    print("1st API finish")
  
async def async_api2():
    print("call 2st API")
    await asyncio.sleep(3)
    print("2nd API finish")
    
start = time.time()
# await async_api1()
# await async_api2()
await asyncio.gather(async_api1(), async_api2())
print(time.time()-start)

call 1st API
call 2st API
2nd API finish
1st API finish
4.003184795379639


## Use case: requests

In [6]:
import time
import json
import os
import requests  
from requests.exceptions import HTTPError


GOOGLE_BOOKS_URL = "https://www.googleapis.com/books/v1/volumes?q=isbn:"
LIST_ISBN = [
    '9780002005883',
    '9780002238304',
    '9780002261982',
    '9780006163831',
    '9780006178736',
    '9780006280897',
    '9780006280934',
    '9780006353287',
    '9780006380832',
    '9780006470229',
]


def extract_fields_from_response(item):
    """Extract fields from API's response"""
    volume_info = item.get("volumeInfo", {})
    title = volume_info.get("title", None)
    subtitle = volume_info.get("subtitle", None)
    description = volume_info.get("description", None)
    published_date = volume_info.get("publishedDate", None)
    return (
        title,
        subtitle,
        description,
        published_date,
    )


def get_book_details_seq(isbn, session):
    """Get book details using Google Books API (sequentially)"""
    url = GOOGLE_BOOKS_URL + isbn
    response = None
    try:
        response = session.get(url)
        response.raise_for_status()
        print(f"Response status ({url}): {response.status_code}")
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error ocurred: {err}")
    response_json = response.json()
    items = response_json.get("items", [{}])[0]
    return items


start = time.time()
with requests.Session() as session:
    for isbn in LIST_ISBN:
        try:
            response = get_book_details_seq(isbn, session)
            parsed_response = extract_fields_from_response(response)
            print(f"Response: {json.dumps(parsed_response, indent=2)}")
        except Exception as err:
            print(f"Exception occured: {err}")
            pass
print(time.time()-start)

Response status (https://www.googleapis.com/books/v1/volumes?q=isbn:9780002005883): 200
Response: [
  "Gilead",
  null,
  "A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It\u2019s 1956 in Gilead, Iowa, towards the end of the Reverend Ames\u2019s life, and he is absorbed in recording his family\u2019s story, a legacy for the young son he will never see grow up. Haunted by his grandfather\u2019s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend\u2019s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames\u2019s joyous, rambling voice that

In [8]:
!pip install aiohttp

     -------------------------------------- 554.9/554.9 KB 1.1 MB/s eta 0:00:00
     -------------------------------------- 122.1/122.1 KB 3.6 MB/s eta 0:00:00


In [9]:
import aiohttp
import asyncio
import os

from aiohttp import ClientSession


GOOGLE_BOOKS_URL = "https://www.googleapis.com/books/v1/volumes?q=isbn:"
LIST_ISBN = [
    '9780002005883',
    '9780002238304',
    '9780002261982',
    '9780006163831',
    '9780006178736',
    '9780006280897',
    '9780006280934',
    '9780006353287',
    '9780006380832',
    '9780006470229',
]


def extract_fields_from_response(response):
    """Extract fields from API's response"""
    item = response.get("items", [{}])[0]
    volume_info = item.get("volumeInfo", {})
    title = volume_info.get("title", None)
    subtitle = volume_info.get("subtitle", None)
    description = volume_info.get("description", None)
    published_date = volume_info.get("publishedDate", None)
    return (
        title,
        subtitle,
        description,
        published_date,
    )


async def get_book_details_async(isbn, session):
    """Get book details using Google Books API (asynchronously)"""
    url = GOOGLE_BOOKS_URL + isbn
    try:
        response = await session.request(method='GET', url=url)
        response.raise_for_status()
        print(f"Response status ({url}): {response.status}")
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error ocurred: {err}")
    response_json = await response.json()
    return response_json


async def run_program(isbn, session):
    """Wrapper for running program in an asynchronous manner"""
    try:
        response = await get_book_details_async(isbn, session)
        parsed_response = extract_fields_from_response(response)
        print(f"Response: {json.dumps(parsed_response, indent=2)}")
    except Exception as err:
        print(f"Exception occured: {err}")
        pass

start = time.time()
async with ClientSession() as session:
    await asyncio.gather(*[run_program(isbn, session) for isbn in LIST_ISBN])
print(time.time()-start)

Response status (https://www.googleapis.com/books/v1/volumes?q=isbn:9780002261982): 200
Response: [
  "Spider's Web",
  "A Novel",
  "A new 'Christie for Christmas' -- a full-length novel adapted from her acclaimed play by Charles Osborne Following BLACK COFFEE and THE UNEXPECTED GUEST comes the final Agatha Christie play novelisation, bringing her superb storytelling to a new legion of fans. Clarissa, the wife of a Foreign Office diplomat, is given to daydreaming. 'Supposing I were to come down one morning and find a dead body in the library, what should I do?' she muses. Clarissa has her chance to find out when she discovers a body in the drawing-room of her house in Kent. Desperate to dispose of the body before her husband comes home with an important foreign politician, Clarissa persuades her three house guests to become accessories and accomplices. It seems that the murdered man was not unknown to certain members of the house party (but which ones?), and the search begins for the 

In [3]:
GOOGLE_BOOKS_URL = "https://www.googleapis.com/books/v1/volumes?q=isbn:"
LIST_ISBN = [
    '9780002005883',
    '9780002238304',
    '9780002261982',
    '9780006163831',
    '9780006178736',
    '9780006280897',
    '9780006280934',
    '9780006353287',
    '9780006380832',
    '9780006470229',
]

def request_single_book(url):
    resp = requests.get(url)

import threading
import requests
import time
start = time.time()
threads = []
for isbn in LIST_ISBN:
    t = threading.Thread(target=request_single_book, args=(GOOGLE_BOOKS_URL+isbn,))
    t.start()
    threads.append(t)
for t in threads:
    t.join()
print(time.time()-start)

0.9517850875854492


In [ ]:
from multiprocessing import Pool

pool = Pool(processes=5)

urls = []
for isbn in LIST_ISBN:
    urls.append(GOOGLE_BOOKS_URL+isbn)

start = time.time()
results = pool.map(request_single_book, urls)
print(time.time()-start)